# Extrahieren von Text zum Social Media Archiv 
- https://democrats-intelligence.house.gov/social-media-content/
- ```pip install progressbar```

In [27]:
import textract
import re
import os
import pandas as pd
import progressbar
import time

## Mit einer Datei ausprobieren

- Einleseversuch

In [28]:
#Testing with one file
text = textract.process('files/P(1)0000054.pdf', method='pdfminer')
text = str(text).replace("\n\n", " ")
text = str(text).replace("\n", " ")
text = str(text).replace("\\n", " ")
text

"b'Ad ID  374  Ad Text  Join us because we care. Black matters.  Ad Landing Page  https://www.facebook.com/Black-Matters-1579673598947501 /  Ad Targeting  Location: United States: Baltimore (+20 km) Maryland; St. Louis (+20 km) Missouri Excluded Connections: Exclude people who like Black Matters Age: 18 - 65+ Language: English (UK) or English (US) Placements: News Feed on desktop computers or News Feed on mobile devices  Ad Impressions  137  Ad Clicks  0  Ad Spend  44.87 RUB  Ad Creation Date  06/10/15 02:59:53 AM PDT  P(1)0000054  Redactions Completed at the Direction of Ranking Member of the US House Permanent Select Committee on Intelligence\\x0cSuggested Page  Black Matters Sponsored  Join us because we care. Black matters.  Black Matters Commun,ty 224,537 people like  nis.  ,` Like Page  P(1)0000055  Redactions Completed at the Direction of Ranking Member of the US House Permanent Select Committee on Intelligence\\x0c'"

- Suchen nach dem Adtext:

In [29]:
adtext = re.search("Ad Text.*Ad Landing Page", text)
adtext = re.search("Ad Text.*Ad Landing Page", text).group().replace("Ad Landing Page", "")
adtext = adtext.replace("Ad Text", "").replace("\\n", " ").strip()
adtext

'Join us because we care. Black matters.'

In [30]:
def get_location(text):
    if re.search("Location.*(Age)", text) != None:
        pattern = re.search("Location.*(Age)", text)
        loc = pattern.group().replace('Age','').replace('Location - Living In: ','')
    else:
        loc = 'N/A'
    return loc
get_location(text)

'Location: United States: Baltimore (+20 km) Maryland; St. Louis (+20 km) Missouri Excluded Connections: Exclude people who like Black Matters '

In [31]:
def get_date(text):
    if re.search("Creation Date.*(Redactions)", str(text)) != None:
        pattern = re.search("Creation Date.*(Redactions)", str(text))
        pattern = pattern.group().replace('Creation Date\\n\\n','')
        date = re.search("\d\d/\d\d/\d\d \d\d:\d\d:\d\d [A-Z][A-Z] [A-Z]{3}", pattern)
        if date != None:
            date = date.group()
    else: 
        date = 'N/A'
    return date
get_date(text)

'06/10/15 02:59:53 AM PDT'

In [32]:
def get_ad_klicks(text):
    if re.search("Ad Clicks.*", text) != None:
        klicks = re.search("Ad Clicks.*", text)
        klicks = klicks.group()[:25].replace(",", "")
        klicks = re.search("[0-9]+", klicks)
        if klicks != None:
            klicks = klicks.group()
    else:
        klicks = 'N/A'
    return klicks
get_ad_klicks(text)

'0'

In [33]:
def get_ad_impressions(text):
    if re.search("Ad Impressions.*", text) != None:
        imps = re.search("Ad Impressions.*", text)
        imps = imps.group()[:25].replace(",", "")
        imps = re.search("[0-9]+", imps)
        if imps != None:
            imps = imps.group()
    else:
        imps = 'N/A'
    return imps
get_ad_impressions(text)

'137'

In [34]:
def get_ad_spend(text):
    if re.search("Ad Spend.*", text) != None:
        adspend = re.search("Ad Spend.*", text)
        adspend = adspend.group()[:25].replace(",", "")
        adspend = re.search("[0-9]+.[0-9]+", adspend)
        if adspend != None:
            adspend = adspend.group()
    else:
        adspend = 'N/A'
    return adspend
get_ad_spend(text)

'44.87'

## Aufbau der Dateiliste

In [35]:
!ls

2.1 Importing Text from Everywhere.ipynb
2.2 Real world example with textract.ipynb
2.3 Named Entities Recognition.ipynb
2.4 Classifying Text.ipynb
2.5 OpenCV - Gesichtserkennung.ipynb
2.6 Document Similarity with TDF-IDF.ipynb
2.7 [OPTIONAL] Similar Words and Sentences with spaCy.ipynb
2.8 [OPTIONAL] Objekterkennung as a service mit IBM Vision.ipynb
2.9 [OPTIONAL] Sentiment, Spracherkennung, Spellchecking.ipynb
bild.jpg
example.docx
example.pdf
example.png
files
fruitbowl.jpg
godfather.txt
guardian.png
haarcascades
instagram
obama.txt
readable.csv
schindlers_list.txt
shawnshank.txt
songtexte
tf-idf.jpeg


In [36]:
#Creating file list
lt = sorted(os.listdir("files"))

In [37]:
lst = []
for elem in lt:
    if ".pdf" in elem:
        lst.append(elem)

In [38]:
#lst = lst[0:4]

In [44]:
#Creating dictionary for:

#'Location', 'Number', 'Date', 'Interests',
#'Age', 'Language', 'Placements', 'Ad Clicks',
#'Ad Impressions', 'Ad Spend'

bar = progressbar.ProgressBar()
results = []

for elem,i in zip(lst, bar(range(len(lst)-1))):
    #print(elem)
    if 'ultimate_names.csv' in elem:
        continue
    
    else:
        
        text = textract.process('files/'+elem, method='pdfminer')
        text = str(text).replace("\n\n", " ")
        text = str(text).replace("\n", " ")

    
        location = get_location(text)
        date = get_date(text)
        klicks = get_ad_klicks(text)
        imps = get_ad_impressions(text)
        adspend = get_ad_spend(text)

        
        mini_dict = {'Location': location,
                 'Number': elem,
                 'Date': date,
                 'Interests': interests,
                 'Age': age,
                 'Language': lang,
                 'Placements': place,
                 'Ad Text': adtext,
                 'Ad Clicks': klicks,
                 'Ad Impressions': imps,
                 'Ad Spend': adspend}
        
        results.append(mini_dict)


100% |########################################################################|


Version von Barney mit if else

In [43]:
#Creating dictionary for:

#'Location', 'Number', 'Date', 'Interests',
#'Age', 'Language', 'Placements', 'Ad Clicks',
#'Ad Impressions', 'Ad Spend'

bar = progressbar.ProgressBar()
results = []

for elem,i in zip(lst, bar(range(len(lst)-1))):
    #print(elem)
    if 'ultimate_names.csv' in elem:
        continue
    
    else:
        
        text = textract.process('files/'+elem, method='pdfminer')
        text = str(text).replace("\n\n", " ")
        text = str(text).replace("\n", " ")

        
        
        if re.search("Interests:.*", text) != None:
            interests = re.search("Interests:.*", text).group().split('\\n')[0]
        else:
            interests = 'N/A'
    
        if re.search("Age:.*", text) != None:
            age = re.search("Age:.*", text).group().split('\\n')[0]
        else:
            age = 'N/A'
        
        if re.search("Language:.*", text) != None:
            lang = re.search("Language:.*", text).group().split('\\n')[0]
        else:
            lang = 'N/A'
    
        if re.search("Placements:.*", text) != None:
            place = re.search("Placements:.*", text).group().split('\\n')[0]
        else:
            place = 'N/A'
    
        if re.search("Ad Text.*Ad Landing Page", text) != None:
            adtext = re.search("Ad Text.*Ad Landing Page", text).group().replace("Ad Landing Page", "")
            adtext = adtext.replace("Ad Text", "").replace("\\n", " ").strip()
        else:
            place = 'N/A'
        
        mini_dict = {'Location': location,
                 'Number': elem,
                 'Date': date,
                 'Interests': interests,
                 'Age': age,
                 'Language': lang,
                 'Placements': place,
                 'Ad Text': adtext,
                 'Ad Clicks': klicks,
                 'Ad Impressions': imps,
                 'Ad Spend': adspend}
        
        results.append(mini_dict)


100% |########################################################################|


### Abspeichern im Dataframe

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df

### Abspeichern als CSV

In [ ]:
df.to_csv('readable.csv')